In [11]:
# connect to Chromadb
import json, os, chromadb
from dotenv import load_dotenv

load_dotenv()
DB_PATH = os.getenv("DB_PATH")


In [16]:
def list_collections(name_of_collection):
    collections = chromadb.PersistentClient(path=DB_PATH).get_collection(name=name_of_collection)
    # note that get() is used on the collection object to get the items within the collection
    info = {}
    info['name'] = collections.name
    info['first_10'] = collections.peek()
    info['num_of_items'] = collections.count()
    return info

In [17]:
for key, value in list_collections("T074T5Q38RW").items():
        print(f"{key}: {value}")

name: T074T5Q38RW
first_10: {'ids': ['T074T5Q38RW'], 'embeddings': [[-0.25021839141845703, -0.44636040925979614, -0.3443649709224701, 0.21280154585838318, 0.27668625116348267, -0.541593074798584, 0.15184365212917328, 0.7818597555160522, -0.18389929831027985, 0.059793002903461456, 0.23707100749015808, -0.5001236796379089, -0.08004219830036163, 0.20055833458900452, -0.5737999081611633, -0.045047029852867126, 0.004770404193550348, -0.5738748908042908, 0.17521581053733826, -0.4359094500541687, -0.2565724849700928, -0.5228275060653687, 0.06438355147838593, 0.032626960426568985, -0.27442747354507446, 0.3842592239379883, -0.3433743119239807, 0.4071587920188904, 0.22938795387744904, -0.8675217628479004, 0.16134999692440033, 0.7152584791183472, 0.435170978307724, 0.31949320435523987, -0.07656960189342499, -0.11166208982467651, 0.47190165519714355, -0.13405966758728027, 0.2372773438692093, -0.09607747197151184, 0.009506374597549438, -0.5298641920089722, 0.7680280804634094, -0.29544663429260254, 